# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import seaborn as sns
import re, string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data loading

In [ ]:
data_path = '/content/drive/MyDrive/ZIwG P/citations.csv'
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 120)

df = pd.read_csv(data_path)
df = df.dropna()
df = df.reset_index(drop=True)
# df.info()


df['label'].replace({'Fałsz': 0, 'Prawda': 1, 'Manipulacja': 2, 'Nieweryfikowalne': 3}, inplace=True)
# X.head()
# y.head(10)
df

,content,author,label
0,O ile my podnieśliśmy pensję minimalną w czasie 8 lat? (…) o ponad 90 proc.,Izabela Leszczyna,0
1,W rankingu praworządności World Justice Project w 2020 r. Polska znów spadła na niższą pozycję.,Hanna Gill-Piątek,1
2,"Na Uniwersytecie Warszawskim powstał taki raport, który jest oczywiście państwu znany, z którego wyraźnie wynika, że...",Dariusz Rosati,1
3,Średnia emerytura w Polsce kobiet i mężczyzn różni się aż o 1000 zł.,Małgorzata Kidawa-Błońska,1
4,"Proszę mi pokazać (…) jedną osobę skazaną przez Trybunał Stanu od czasu, kiedy Trybunał Stanu w Polsce, wolnej, tak ...",Paweł Kukiz,0
...,...,...,...
4332,"Jeżeli chodzi o penalizację homoseksualizmu, to w Polsce nigdy czegoś takiego nie było. Ani za I Rzeczypospolitej, a...",Kosma Złotowski,0
4333,Rządowy Fundusz Inwestycji Lokalnych na Pomorzu – największy beneficjent gmina miasta Gdańsk.,Marcin Horała,1
4334,"Zarówno WHO, EMA, czyli Europejska Agencja Leków, jak i polskie instytucje, w tym Urząd Rejestracji Produktów Leczni...",Michał Dworczyk,1
4335,"W żadnym z tych krajów (które zawiesiły stosowanie szczepionki AstraZeneca – przyp. Demagog) nie ustalono, że taki z...",Michał Dworczyk,1


# Text preprocessing

In [ ]:
def remove_punctuation(text):
    no_punctuation_text = "".join([char for char in text if char not in string.punctuation])
    return no_punctuation_text


def tokenize(text):
    tokens = re.split('\W+', text)
    text = [word for word in tokens]
    return text


def lemmatizing(text):
    #TODO when we get polish lemmatizer
    pass


#TODO when we decide what how to preprocess text
# now it is just copy-paste from online course
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens]
    return text

In [ ]:
df['clean_text'] = df['content'].apply(lambda x: clean_text(x))
df.head()

,content,author,label,clean_text
0,O ile my podnieśliśmy pensję minimalną w czasie 8 lat? (…) o ponad 90 proc.,Izabela Leszczyna,0,"[o, ile, my, podnieśliśmy, pensję, minimalną, w, czasie, 8, lat, o, ponad, 90, proc]"
1,W rankingu praworządności World Justice Project w 2020 r. Polska znów spadła na niższą pozycję.,Hanna Gill-Piątek,1,"[w, rankingu, praworządności, world, justice, project, w, 2020, r, polska, znów, spadła, na, niższą, pozycję]"
2,"Na Uniwersytecie Warszawskim powstał taki raport, który jest oczywiście państwu znany, z którego wyraźnie wynika, że...",Dariusz Rosati,1,"[na, uniwersytecie, warszawskim, powstał, taki, raport, który, jest, oczywiście, państwu, znany, z, którego, wyraźni..."
3,Średnia emerytura w Polsce kobiet i mężczyzn różni się aż o 1000 zł.,Małgorzata Kidawa-Błońska,1,"[średnia, emerytura, w, polsce, kobiet, i, mężczyzn, różni, się, aż, o, 1000, zł]"
4,"Proszę mi pokazać (…) jedną osobę skazaną przez Trybunał Stanu od czasu, kiedy Trybunał Stanu w Polsce, wolnej, tak ...",Paweł Kukiz,0,"[proszę, mi, pokazać, jedną, osobę, skazaną, przez, trybunał, stanu, od, czasu, kiedy, trybunał, stanu, w, polsce, w..."


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size=0.2)

## Vectorization

### N-gram

In [ ]:
# range = (2, 2)

# ngram_vect = CountVectorizer(ngram_range=range)
# X_ngram = ngram_vect.fit_transform(X['content'])

# X_ngram_df = pd.DataFrame(X_ngram.toarray())
# X_ngram_df.columns = ngram_vect.get_feature_names()
# X_ngram_df

### TF-IDF

In [ ]:
# tfidf_vect = TfidfVectorizer()
# tfidf_vect.fit(X_train['content'])

# X_train_vect = tfidf_vect.transform(X_train['content'])
# X_test_vect = tfidf_vect.transform(X_test['content'])

In [ ]:
# X_test_vect[0].toarray()

# Model nr 1

# Model nr 2

# Deep Learning

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_seq_pad = pad_sequences(X_train_seq, 5000)
X_test_seq_pad = pad_sequences(X_test_seq, 5000)



## RNN model

In [ ]:
model = Sequential()

model.add(Embedding(len(tokenizer.index_word)+1, 32))
model.add(LSTM(32, dropout=0, recurrent_dropout=0))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          491808    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 501,217
Trainable params: 501,217
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_seq_pad, y_train, validation_data=(X_test_seq_pad, y_test), batch_size=64, epochs=20)

Epoch 1/20
55/55 [==============================] - 247s 4s/step - loss: 0.0000e+00 - accuracy: 0.5912 - val_loss: 0.0000e+00 - val_accuracy: 0.5829
Epoch 2/20
55/55 [==============================] - 255s 5s/step - loss: 0.0000e+00 - accuracy: 0.5861 - val_loss: 0.0000e+00 - val_accuracy: 0.5829
Epoch 3/20
55/55 [==============================] - 257s 5s/step - loss: 0.0000e+00 - accuracy: 0.5910 - val_loss: 0.0000e+00 - val_accuracy: 0.5829
Epoch 4/20
55/55 [==============================] - 258s 5s/step - loss: 0.0000e+00 - accuracy: 0.5921 - val_loss: 0.0000e+00 - val_accuracy: 0.5829
Epoch 5/20
55/55 [==============================] - 256s 5s/step - loss: 0.0000e+00 - accuracy: 0.5949 - val_loss: 0.0000e+00 - val_accuracy: 0.5829
Epoch 6/20
55/55 [==============================] - 257s 5s/step - loss: 0.0000e+00 - accuracy: 0.5971 - val_loss: 0.0000e+00 - val_accuracy: 0.5829
Epoch 7/20
55/55 [==============================] - 257s 5s/step - loss: 0.0000e+00 - accuracy: 0.5922 - v

In [27]:
# X_train_seq_pad[0]

prediction = model.predict(X_train_seq_pad[50])
prediction
# predictions = np.argmax(model.predict(test_image_gen), axis=-1)

# plt.figure(figsize=(10,6))
# sns.heatmap(confusion_matrix(test_image_gen.classes,predictions),annot=True)

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

# Evaluation